In [62]:
import tarfile
import urllib
#import urllib2
from os.path import isfile, isdir
from tqdm import tqdm

import os
#import cv2
import h5py
import math
import numpy as np
from PIL import Image
from scipy import misc
import random as rand
from operator import add
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches
import tarfile
import urllib
from IPython.display import display, Image
from scipy import ndimage
#import cPickle as pickle
import scipy.io as sio
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import sys
import pandas as pd
%matplotlib inline
from sklearn.model_selection import train_test_split
import glob
import csv

import re
#import nltk
#from nltk.corpus import stopwords
#import nltk.data
from sklearn.utils import shuffle
import zipfile

In [63]:
train_full = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Clean data

In [92]:
def clean_question(question):
    """remove non-letters, return a list of words"""
    if type(question) != float:
        question =  re.sub('[^a-zA-Z]', " ", question)
        return question.lower().split()
    else:
        return ""

In [93]:
def clean_questions(questions):
    """clean questions that are not empty, return a list of lists of words"""
    new_questions = []   
        
    for idx, question in enumerate(questions):        
        #if (idx+1)%100000 == 0:
            #print("Review %d of %d\n" % ( idx+1, len(questions) ))
        
        new_questions.append(clean_question(question))

    return new_questions

In [94]:
train_full_y = train_full['is_duplicate']
train_full_q1 = clean_questions(train_full['question1'])
train_full_q2 = clean_questions(train_full['question2'])
test_q1 = clean_questions(test['question1'])
test_q2 = clean_questions(test['question2'])

In [67]:
# testing
print(len(train_full_q1),len(train_full_q2), len(train_full_y))
print("match records: ", np.count_nonzero(train_full_y)/len(train_full_y) )
print("non_match records: ", 1-np.count_nonzero(train_full_y)/len(train_full_y) )
print(len(test_q1), len(test_q2))

404290 404290 404290
match records:  0.369197853026293
non_match records:  0.6308021469737071
2345796 2345796


In [7]:
# distribution of the questions' length
questions_len = []
for q in train_full_q1 + train_full_q2 + test_q1 + test_q2:
    questions_len.append(len(q))

print("Distribution of the questions' length")

print("max:", np.amax(np.array(questions_len)), 
      "; 95%: ", np.percentile(np.array(questions_len), 95),
      "; 92%: ", np.percentile(np.array(questions_len), 92),
      "; 90%: ", np.percentile(np.array(questions_len), 90),
      "; 80%: ", np.percentile(np.array(questions_len), 80),
      "; median: ", np.percentile(np.array(questions_len), 50))

Distribution of the questions' length
max: 246 ; 95%:  23.0 ; 92%:  20.0 ; 90%:  18.0 ; 80%:  14.0 ; median:  10.0


# Augment datasets

In [68]:
from itertools import chain
from nltk.corpus import wordnet

In [69]:
def get_synonyms(word):
    synonyms = wordnet.synsets(word)
    return list(set(chain.from_iterable([word.lemma_names() for word in synonyms])))

In [40]:
get_synonyms("you")

[]

In [70]:
import random

def augment_question(question, k):
    words_to_change = random.sample(question, int(k*len(question)))
    for idx, word in enumerate(question):
        if word in words_to_change:
            synonyms = get_synonyms(word)
            if synonyms != []:
                synonym = random.choice(synonyms)
                question[idx] = synonym
    return question

In [76]:
q1 = train_full_q1[1000]
print(q1)

['how', 'can', 'i', 'convert', 'raw', 'files', 'to', 'jpeg', 'in', 'photos', 'in', 'a', 'macbook']


In [78]:
q1a = augment_question(q1, 0.5)

In [43]:
augment_question(['i', 'love', 'you', 'please', 'help'], 0.5)

['i', 'sleep_with', 'you', 'please', 'help']

In [82]:
def augment_dataset(dataset):
    if type(dataset[0][1]) == str:
        for question in dataset:
            new_question = augment_question(question, 0.5)            
            dataset.append(new_question)   
        return dataset
    else:
        return dataset+ dataset

In [83]:
augment_dataset(train_full_q1)

KeyboardInterrupt: 

In [11]:
for dataset in [train_full_q1, train_full_q2, train_full_y]:
    dataset = augment_dataset(dataset)

# Prepare datasets

In [12]:
train_q1, cv_q1, train_q2, cv_q2, train_y, cv_y = train_test_split(train_full_q1, train_full_q2, train_full_y, test_size = 0.2, random_state=3)

In [13]:
train_q1, train_q2, train_y = shuffle(train_q1, train_q2, train_y, random_state=3)
cv_q1, cv_q2, cv_y = shuffle(cv_q1, cv_q2, cv_y, random_state=3)

In [14]:
print(len(train_q1), len(train_q2), len(train_y), len(cv_q1), len(cv_q2), len(cv_y))

323432 323432 323432 80858 80858 80858


In [15]:
np.save('data/train_q1', train_q1)
np.save('data/train_q2', train_q2)
np.save('data/train_y', train_y)
np.save('data/cv_q1', cv_q1)
np.save('data/cv_q2', cv_q2)
np.save('data/cv_y', cv_y)
np.save('data/test_q1', test_q1)
np.save('data/test_q2', test_q2)